# RTK Processing of EMLID Reach RS+ GPS data

**Script prepared by A. Rovere - MARUM, University of Bremen**

This script can be used to process data from two EMLID REACH RS+ GPS receivers (single band). One is a rover, the second is used as base station. At the time of data collection, the two receivers are connected through radio, working in RTK. Base station data are first acquired "on the fly" with a 10-minutes average single measurement, and then postprocessed using wither PPP or a PPK. To use the script, the following inputs are required:

 - Initial base station position
 - Corrected Lat,Lon,Height of base station
 - Files exported from the data collector in *.csv* format

The script first merges the data collector files into a single dataframe. Then, postprocessed rover data is imported and a new dataframe is created with time-averaged postprocessed static positions acquired in FIX status. Time-averaged positions are also calculated for FLOAT status datapoints. All the results are saved in a multi-sheet excel file.

A general outline of the survey workflow is explained here:

https://docs.emlid.com/reach/common/tutorials/placing-the-base/

In [7]:
#Packages needed
import pandas as pd
import glob
import os
from pathlib import Path
import xlsxwriter as writer
import numpy as np

## Input folders and data
For the script to work, it is necessary to indicate the folders where data are saved and some of the outputs from the NRCAN processing. See instructions below. 

In [8]:
#INSERT THE FOLLOWING VALUES
#Output file name
out='Example_output.xlsx'
#Insert the base station name
station_name = 'Caleta Olivia base 2 November 2012' 
#Folder where CSV files from the data collector are stored
csv_folder = r'\Example_data\Rover\Data_collector'
#Folder where results of NRCAN processing are stored
processed_data = r'\Example_data\Rover\Processed'

# Insert base station values
Hereafter, insert the base station data. Two sets of values are needed.
 - Base station coordinates as averaged in the field.
 - Base station coordinates after postprocessing.

In [9]:
# Coordinates of base station as averaged in the field
Lat=-46.5581074598 #dec degrees
Lon=-67.4328075686 #dec degrees
H=34.053 #m
Ant_Height=1.305 #m

In [10]:
# Postprocessed coordinates of base station
corr_Lat='46-33-29.25944S' #degrees, minutes, seconds 
corr_Lon='67-25-58.14884W' #degrees, minutes, seconds
corr_H=34.439 #m

sigma_Lat=0.389 #m
sigma_Lon=0.503 #m
sigma_H=0.680 #m

#DO NOT MODIFY THE PART BELOW
def convert(tude):
    multiplier = 1 if tude[-1] in ['N', 'E'] else -1
    return multiplier * sum(float(x) / 60 ** n for n, x in enumerate(tude[:-1].split('-')))
#Benchmark location
corr_Lat=convert(corr_Lat)
corr_Lon=convert(corr_Lon)

In [11]:
#Write base station data to excel
#Create dataframe
BaseData=pd.DataFrame({'Latitude (decimal degrees)':[Lat,corr_Lat],
        'Longitude (decimal degrees)':[Lon,corr_Lon],
        'Antenna height above Ellipsoid (m)':[H, corr_H],
        'Postprocessed Latitude 2sigma (m)':['0',sigma_Lat],
        'Postprocessed Longitude 2sigma (m)':['0',sigma_Lon],
        'Postprocessed Ellipsoid height 2sigma (m)':['0',sigma_H]},
           index=['Field position', 'Postprocessed position']).T

#Calculate deltas
BaseData['Deltas']=BaseData['Postprocessed position'].astype(float) - BaseData['Field position'].astype(float)

#Open Excel workbook
writer = pd.ExcelWriter(out, engine='xlsxwriter',
                        options={'strings_to_numbers': True,
                                 'strings_to_formulas': False})
workbook=writer.book
wrap = workbook.add_format({'text_wrap': True, 
                            'valign':'vcenter',
                            'align':'center'})

BaseData.to_excel(writer, sheet_name='Base Station data', index=True)
worksheet = writer.sheets['Base Station data']
worksheet.set_column('A:ZZ',20,wrap)
header_format = workbook.add_format({'bold': True,'text_wrap': True,'valign': 'vcenter','align':'center',
                                     'fg_color':'#C0C0C0','border': 1})
for col_num, value in enumerate(BaseData.columns.values):
    worksheet.write(0, col_num+1, value, header_format)

BaseData

,Field position,Postprocessed position,Deltas
Latitude (decimal degrees),-46.5581,-46.5581,-0.000020
Longitude (decimal degrees),-67.4328,-67.4328,-0.000012
Antenna height above Ellipsoid (m),34.053,34.439,0.386000
Postprocessed Latitude 2sigma (m),0,0.389,0.389000
Postprocessed Longitude 2sigma (m),0,0.503,0.503000
Postprocessed Ellipsoid height 2sigma (m),0,0.68,0.680000


# Data collector files
Include in one folder all the *.csv* files exported from the data collector (Apple or Android) with ReachView software.

In [13]:
dirname = os.path.realpath('')+csv_folder
all_files = glob.glob(dirname + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['filename'] = os.path.basename(filename)
    li.append(df)
RTKpoints = pd.concat(li, axis=0, ignore_index=True)
RTKpoints['collection start'] = RTKpoints['collection start'].astype('datetime64[ns]')
RTKpoints['collection end'] = RTKpoints['collection end'].astype('datetime64[ns]')
RTKpoints.sort_values(by=['collection start'],inplace=True)
RTKpoints.reset_index(inplace=True)
RTKpoints.rename(columns={'name':'POINT ID',
                          'collection start':'Start of data collection (yyyy-mm-dd UTC)',
                         'collection end':'End of data collection (yyyy-mm-dd UTC)',
                         'sample count':'Total points sampled in the field',
                         'antenna height':'Rover Antenna height (m)',
                         'longitude':'Uncorrected Longitude (decimal degrees)',
                         'latitude':'Uncorrected Latitude (decimal degrees)',
                         'elevation':'Uncorrected ground elevation above ellipsoid (m)'},inplace=True)
RTKpoints.drop(columns=['level_0','index'],inplace=True)

RTKpoints['Corrected Longitude (decimal degrees)']=RTKpoints['Uncorrected Longitude (decimal degrees)']+BaseData['Deltas']['Longitude (decimal degrees)']
RTKpoints['Corrected Latitude (decimal degrees)']=RTKpoints['Uncorrected Latitude (decimal degrees)']+BaseData['Deltas']['Latitude (decimal degrees)']
RTKpoints['Corrected ground elevation above ellipsoid (m)']=RTKpoints['Uncorrected ground elevation above ellipsoid (m)']+BaseData['Deltas']['Antenna height above Ellipsoid (m)']
RTKpoints['Longitude 2sigma (m)']=np.sqrt(np.square(RTKpoints['lateral rms'])+np.square(BaseData['Deltas']['Postprocessed Longitude 2sigma (m)']))
RTKpoints['Latitude 2sigma (m)']=np.sqrt(np.square(RTKpoints['lateral rms'])+np.square(BaseData['Deltas']['Postprocessed Latitude 2sigma (m)']))
RTKpoints['Elevation 2sigma (m)']=np.sqrt(np.square(RTKpoints['lateral rms']*2)+np.square(BaseData['Deltas']['Postprocessed Ellipsoid height 2sigma (m)']))

RTKpoints.to_excel(writer, sheet_name='Rover RTK points', index=False)
worksheet = writer.sheets['Rover RTK points']
worksheet.set_column('A:ZZ',20,wrap)
header_format = workbook.add_format({'bold': True,'text_wrap': True,
                                         'valign': 'vcenter','align':'center',
                                         'fg_color':'#C0C0C0','border': 1})
for col_num, value in enumerate(RTKpoints.columns.values):
    worksheet.write(0, col_num, value, header_format)        
        
workbook.close()

RTKpoints

,POINT ID,Uncorrected Longitude (decimal degrees),Uncorrected Latitude (decimal degrees),Uncorrected ground elevation above ellipsoid (m),Start of data collection (yyyy-mm-dd UTC),End of data collection (yyyy-mm-dd UTC),solution status,lateral rms,Total points sampled in the field,Rover Antenna height (m),filename,Corrected Longitude (decimal degrees),Corrected Latitude (decimal degrees),Corrected ground elevation above ellipsoid (m),Longitude 2sigma (m),Latitude 2sigma (m),Elevation 2sigma (m)
0,Base,-67.432806,-46.558108,33.850066,2019-11-02 15:07:07,2019-11-02 15:09:31,FIX,0.0220,715.0,1.775,2 Nov 2019.csv,-67.432818,-46.558128,34.236066,0.503481,0.389622,0.681422
1,Point 2,-67.432351,-46.557330,13.772488,2019-11-02 15:24:52,2019-11-02 15:27:03,FIX,0.0187,659.0,1.775,2 Nov 2019.csv,-67.432362,-46.557350,14.158488,0.503347,0.389449,0.681028
2,Point 3,-67.432435,-46.557357,14.547374,2019-11-02 15:28:14,2019-11-02 15:30:20,FIX,0.0264,628.0,1.775,2 Nov 2019.csv,-67.432447,-46.557377,14.933374,0.503692,0.389895,0.682047
3,Point 4,-67.432489,-46.557386,15.050385,2019-11-02 15:37:30,2019-11-02 15:39:55,FIX,0.0307,724.0,1.775,2 Nov 2019.csv,-67.432500,-46.557406,15.436385,0.503936,0.390210,0.682766
4,Point 5,-67.432560,-46.557418,15.664075,2019-11-02 15:40:53,2019-11-02 15:42:57,FIX,0.0369,620.0,1.775,2 Nov 2019.csv,-67.432571,-46.557438,16.050075,0.504352,0.390746,0.683993
5,Point 6,-67.432687,-46.557473,16.984450,2019-11-02 15:44:00,2019-11-02 15:46:02,FIX,0.0582,611.0,1.775,2 Nov 2019.csv,-67.432699,-46.557493,17.370450,0.506356,0.393330,0.689891
6,Point 7,-67.432760,-46.557506,18.601614,2019-11-02 15:48:12,2019-11-02 15:50:36,FIX,0.0212,721.0,1.775,2 Nov 2019.csv,-67.432772,-46.557526,18.987614,0.503447,0.389577,0.681321
7,Point 8,-67.432859,-46.557555,19.031623,2019-11-02 15:51:32,2019-11-02 15:53:34,FIX,0.0209,611.0,1.775,2 Nov 2019.csv,-67.432871,-46.557575,19.417623,0.503434,0.389561,0.681284
8,Point 9,-67.432530,-46.557011,12.745002,2019-11-02 15:55:41,2019-11-02 15:57:43,FIX,0.0272,607.0,1.775,2 Nov 2019.csv,-67.432542,-46.557031,13.131002,0.503735,0.389950,0.682173
9,Point 10,-67.432829,-46.557837,24.883342,2019-11-02 16:28:37,2019-11-02 16:30:41,FIX,0.0380,624.0,1.775,2 Nov 2019.csv,-67.432840,-46.557857,25.269342,0.504433,0.390852,0.684234


## Description of outputs
The output of this script is an excel file (*.xslx*) containing two sheets.
 - **Base Station Data.** Base station data, both as surveyed and postprocessed.
 - **Rover RTK points.** RTK points surveyed in the field, updated for new base position and uncertainty.


***
## License
This software is relased under the MIT license.

Copyright 2020 Alessio Rovere

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
***
# Research funding acknowledgments
This script and associated data were created in the framework of the European Reasearch Council Starting Grant WARMCOASTS (Grant Agreement Number 802414), funded under the European Union's Horizon 2020 research and Innovation programme.
***
# How to cite
Citation ref here
***
# Code acknowledgments
https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f
https://kite.com/python/answers/how-to-redirect-print-output-to-a-text-file-in-python
https://stackoverflow.com/questions/41857659/python-pandas-add-filename-column-csv
https://stackoverflow.com/questions/46184239/extract-a-page-from-a-pdf-as-a-jpeg
https://stackoverflow.com/questions/48531843/how-to-remove-rows-of-a-dataframe-based-off-of-data-from-another-dataframe
https://medium.com/better-programming/how-to-convert-latitude-longitude-to-distance-utm-and-geojson-34c982cda40
https://stackoverflow.com/questions/21298772/how-to-convert-latitude-longitude-to-decimal-in-python